In [50]:
import pandas as pd
import os
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
import plotly.graph_objects as go
import ruptures as rpt
from itertools import combinations as comb
from statsmodels.stats import power

from IPython.display import display

In [13]:
folder_path = 'data'
def calculate_mean_for_30s(df):
    df['Time (s)'] = pd.to_datetime(df['Time (s)'], unit='s')
    df.set_index('Time (s)', inplace=True)
    resampled_df = df.resample('30s').mean()
    return resampled_df

def calculate_var_for_30s(df):
    resampled_df = df.resample('30s').var()
    return resampled_df

def read_data(filename):
    accelerometer = pd.read_csv(os.path.join(folder_path, filename, 'Accelerometer.csv'),sep=';')
    gyroscope = pd.read_csv(os.path.join(folder_path, filename,'Gyroscope.csv'),sep=';')
    accelerometer.rename(columns={'Acceleration x (m/s^2)':'accelerometer_x','Acceleration y (m/s^2)':'accelerometer_y','Acceleration z (m/s^2)':'accelerometer_z'}, inplace=True)
    gyroscope.rename(columns={'Gyroscope x (rad/s)':'gyroscope_x','Gyroscope y (rad/s)':'gyroscope_y','Gyroscope z (rad/s)':'gyroscope_z'}, inplace=True)
    accelerometer =accelerometer[9000:-9000]
    gyroscope = gyroscope[9000:-9000]
    accelerometer_mean = calculate_mean_for_30s(accelerometer)
    gyroscope_mean = calculate_mean_for_30s(gyroscope)
    accelerometer_var = calculate_var_for_30s(accelerometer)
    gyroscope_var = calculate_var_for_30s(gyroscope)
    return accelerometer, gyroscope , accelerometer_mean, gyroscope_mean, accelerometer_var, gyroscope_var

def do_test(dataframe1, dataframe2):
    t_stat, p_val = stats.ttest_ind(dataframe1, dataframe2)
    return t_stat, p_val

In [14]:
accelerometer_lucas1, gyroscope_lucas1, accelerometer_lucas1_mean, gyroscope_lucas1_mean, accelerometer_lucas1_var, gyroscope_lucas1_var  = read_data('lucas1')
accelerometer_nick1, gyroscope_nick1, accelerometer_nick1_mean, gyroscope_nick1_mean, accelerometer_nick1_var, gyroscope_nick1_var  = read_data('nick1')
accelerometer_till1, gyroscope_till1, accelerometer_till1_mean, gyroscope_till1_mean, accelerometer_till1_var, gyroscope_till1_var  = read_data('till1')
accelerometer_luisa1, gyroscope_luisa1, accelerometer_luisa1_mean, gyroscope_luisa1_mean, accelerometer_luisa1_var, gyroscope_luisa1_var  = read_data('luisa1')

In [15]:
dict_accel_var = {'accelerometer_lucas1_var': accelerometer_lucas1_var, 'accelerometer_nick1_var': accelerometer_nick1_var, 'accelerometer_till1_var': accelerometer_till1_var, 'accelerometer_luisa1_var': accelerometer_luisa1_var}
dict_accel_mean = {'accelerometer_lucas1_mean': accelerometer_lucas1_mean, 'accelerometer_nick1_mean': accelerometer_nick1_mean, 'accelerometer_till1_mean': accelerometer_till1_mean, 'accelerometer_luisa1_mean': accelerometer_luisa1_mean}
dict_gyro_var = {'gyroscope_lucas1_var': gyroscope_lucas1_var, 'gyroscope_nick1_var': gyroscope_nick1_var, 'gyroscope_till1_var': gyroscope_till1_var, 'gyroscope_luisa1_var': gyroscope_luisa1_var}
dict_gyro_mean = {'gyroscope_lucas1_mean': gyroscope_lucas1_mean, 'gyroscope_nick1_mean': gyroscope_nick1_mean, 'gyroscope_till1_mean': gyroscope_till1_mean, 'gyroscope_luisa1_mean': gyroscope_luisa1_mean}
dict_of_dfs = {'accel_mean': dict_accel_mean, 'gyro_mean':dict_gyro_mean,'accel_var':dict_accel_var, 'gyro_var' :dict_gyro_var}

In [68]:
list_columns_accelerometer = ['accelerometer_x', 'accelerometer_y', 'accelerometer_z']
list_columns_gyroscope = ['gyroscope_x', 'gyroscope_y', 'gyroscope_z']

# die art wie ich hier die results speichere suckt, sollte eigentlich nen table als index haben aber we
results = {}

search_string = 'accelerometer'

for df_name, df in dict_of_dfs.items():
    combinations = list(comb(df, 2))
    for combo in combinations:
        if search_string in combo[0]:
            for column in list_columns_accelerometer:
                (df[combo[0]][column].mean()-df[combo[1]][column].mean())/pd.concat([df[combo[0]][column], df[combo[1]][column]]).std()
                t_stat, p_val = do_test(df[combo[0]][column], df[combo[1]][column])
                results[(combo[0],combo[1],column)] = [t_stat, p_val]
        else:
            for column in list_columns_gyroscope:
                t_stat, p_val = do_test(df[combo[0]][column], df[combo[1]][column])
                results[(combo[0],combo[1],column)] = [t_stat, p_val]

results_df = pd.DataFrame.from_dict(results, orient='index', columns=['T Statistic', 'P Value'])


In [69]:
# Define a list of names and corresponding data sets
names = ['lucas1', 'nick1', 'till1', 'luisa1']
data_sets = [(accelerometer_lucas1_mean, accelerometer_lucas1_var, gyroscope_lucas1_mean, gyroscope_lucas1_var),
             (accelerometer_nick1_mean, accelerometer_nick1_var, gyroscope_nick1_mean, gyroscope_nick1_var),
             (accelerometer_till1_mean, accelerometer_till1_var, gyroscope_till1_mean, gyroscope_till1_var),
             (accelerometer_luisa1_mean, accelerometer_luisa1_var, gyroscope_luisa1_mean, gyroscope_luisa1_var)]

# Iterate over each name and data set
for name, data_set in zip(names, data_sets):
    accelerometer_mean, accelerometer_var, gyroscope_mean, gyroscope_var = data_set
    
    for sensor, data in zip(['accelerometer', 'gyroscope'], [accelerometer_mean, gyroscope_mean]):
        for cnt,axis in enumerate(['x', 'y', 'z']):
            t_stat, p_val = stats.ttest_ind(data[:len(data)//2], data[len(data)//2:])
            results_df.loc[f'({sensor}_{name}_mean, {sensor}_{name}_mean, {sensor}_{axis})'] = float(t_stat[cnt]), float(p_val[cnt])

    for sensor, data in zip(['accelerometer', 'gyroscope'], [accelerometer_var, gyroscope_var]):
        for cnt,axis in enumerate(['x', 'y', 'z']):
            t_stat, p_val = stats.ttest_ind(data[:len(data)//2], data[len(data)//2:])
            results_df.loc[f'({sensor}_{name}_var, {sensor}_{name}_var, {sensor}_{axis})'] = float(t_stat[cnt]), float(p_val[cnt])


In [70]:
pd.set_option('display.float_format', '{:.4f}'.format)
display(results_df)

,T Statistic,P Value
"(accelerometer_lucas1_mean, accelerometer_nick1_mean, accelerometer_x)",47.3376,0.0000
"(accelerometer_lucas1_mean, accelerometer_nick1_mean, accelerometer_y)",-2.9718,0.0048
"(accelerometer_lucas1_mean, accelerometer_nick1_mean, accelerometer_z)",10.5200,0.0000
"(accelerometer_lucas1_mean, accelerometer_till1_mean, accelerometer_x)",45.9698,0.0000
"(accelerometer_lucas1_mean, accelerometer_till1_mean, accelerometer_y)",-8.3831,0.0000
...,...,...
"(accelerometer_luisa1_var, accelerometer_luisa1_var, accelerometer_y)",0.9607,0.3476
"(accelerometer_luisa1_var, accelerometer_luisa1_var, accelerometer_z)",1.1341,0.2695
"(gyroscope_luisa1_var, gyroscope_luisa1_var, gyroscope_x)",1.9526,0.0643
"(gyroscope_luisa1_var, gyroscope_luisa1_var, gyroscope_y)",-0.2633,0.7949


In [19]:
t = power.TTestPower()

In [20]:
t.solve_power(effect_size=0.865871, nobs=None, alpha=0.05, power=0.99, alternative='two-sided')

26.52651185356142

In [60]:
4*3*4

48